***
## [November 2018] - Generating measurement sequence files for motorscanGUI
### Abstract:

MotorscanGUI has the ability to run long and complicated sequences of measurements, but the user interface for defining the scan is painful for anything more than a handful of macros.

Happily, it can also save and load scans from simple .xml files. We can generate such files elsewhere then load them into motorscanGUI to run.

Here are some functions and example sequences to get you started with this.


In [2]:
import numpy as np
from BLOCH_beamline import *

---- BLOCH beamline ----
	Last updated 02.12.2018 

	Latest change: M1 pitch lookup



***
## Local functions

In [3]:
def setEPUgap(file,gap):
    file.write('  <macro name="mv">\n')   
    file.write('    <paramrepeat name="motor_pos_list">\n') 
    file.write('      <repeat nr="1">')  
    file.write('        <param name="motor" value="EPU_R1_110_GAP"/>\n')
    file.write('        <param name="pos" value="{0:.1f}"/>\n'.format(gap))
    file.write('      </repeat>\n')
    file.write('    </paramrepeat>\n')
    file.write('  </macro>\n')
    
def setEPUphase(file,phase):
    file.write('  <macro name="mv">\n')   
    file.write('    <paramrepeat name="motor_pos_list">\n') 
    file.write('      <repeat nr="1">')  
    file.write('        <param name="motor" value="EPU_R1_110_PHASE"/>\n')
    file.write('        <param name="pos" value="{0:.1f}"/>\n'.format(phase))
    file.write('      </repeat>\n') 
    file.write('    </paramrepeat>\n')
    file.write('  </macro>\n')
    
def setM1Pitch(file,pitch):
    file.write('  <macro name="mv">\n')   
    file.write('    <paramrepeat name="motor_pos_list">\n') 
    file.write('      <repeat nr="1">')  
    file.write('        <param name="motor" value="m1_pitch"/>\n')
    file.write('        <param name="pos" value="{0:.1f}"/>\n'.format(pitch))
    file.write('      </repeat>\n')
    file.write('    </paramrepeat>\n')
    file.write('  </macro>\n')
    
def setM4Diagnostick(file,position):
    file.write('  <macro name="mv">\n')   
    file.write('    <paramrepeat name="motor_pos_list">\n') 
    file.write('      <repeat nr="1">')  
    file.write('        <param name="motor" value="a_m4_bpm_v"/>\n')
    file.write('        <param name="pos" value="{0:.1f}"/>\n'.format(position))
    file.write('      </repeat>\n')
    file.write('    </paramrepeat>\n')
    file.write('  </macro>\n')
    
def centerMonoBaffles(file):
    file.write('  <macro name="mv">\n')   
    file.write('    <paramrepeat name="motor_pos_list">\n') 
    file.write('      <repeat nr="1">')  
    file.write('        <param name="motor" value="mono_baff_v_offset"/>\n')
    file.write('        <param name="pos" value="-0.8"/>\n')
    file.write('      </repeat>\n')
    file.write('      <repeat nr="1">')  
    file.write('        <param name="motor" value="mono_baff_h_offset"/>\n')
    file.write('        <param name="pos" value="-0.2"/>\n')
    file.write('      </repeat>\n')
    file.write('    </paramrepeat>\n')
    file.write('  </macro>\n')

def setMonoBaffleAperture(file,gapsize):
    v_gap_offset = 3
    h_gap_offset = 2.8
    
    file.write('  <macro name="mv">\n')   
    file.write('    <paramrepeat name="motor_pos_list">\n') 
    file.write('      <repeat nr="1">')  
    file.write('        <param name="motor" value="mono_baff_v_gap"/>\n')
    file.write('        <param name="pos" value="{0}"/>\n'.format(gapsize+v_gap_offset))
    file.write('      </repeat>\n')
    file.write('      <repeat nr="1">')  
    file.write('        <param name="motor" value="mono_baff_h_gap"/>\n')
    file.write('        <param name="pos" value="{0}"/>\n'.format(gapsize+h_gap_offset))
    file.write('      </repeat>\n')
    file.write('    </paramrepeat>\n')
    file.write('  </macro>\n')

def setFEMask_to_3pt5_mm(file):
    file.write('  <macro name="mv">\n')   
    file.write('    <paramrepeat name="motor_pos_list">\n') 
    file.write('      <repeat nr="1">')  
    file.write('        <param name="motor" value="mm1_x"/>\n')
    file.write('        <param name="pos" value="-4"/>\n')
    file.write('      </repeat>\n')

    file.write('      <repeat nr="1">')  
    file.write('        <param name="motor" value="mm2_x"/>\n')
    file.write('        <param name="pos" value="-4.61"/>\n')
    file.write('      </repeat>\n')

    file.write('      <repeat nr="1">')  
    file.write('        <param name="motor" value="mm1_y"/>\n')
    file.write('        <param name="pos" value="-2.8"/>\n')
    file.write('      </repeat>\n')

    file.write('      <repeat nr="1">')  
    file.write('        <param name="motor" value="mm2_y"/>\n')
    file.write('        <param name="pos" value="-4.44"/>\n')
    file.write('      </repeat>\n')
    
    file.write('    </paramrepeat>\n')
    file.write('  </macro>\n')
    
def setExitSlit(file,hgap,vgap):
    file.write('  <macro name="mv">\n')   
    file.write('    <paramrepeat name="motor_pos_list">\n') 
    file.write('      <repeat nr="1">')  
    file.write('        <param name="motor" value="a_slit1_v_gap"/>\n')
    file.write('        <param name="pos" value="{0:.1f}"/>\n'.format(vgap))
    file.write('      </repeat>\n')
    file.write('      <repeat nr="1">')  
    file.write('        <param name="motor" value="a_slit_h_gap"/>\n')
    file.write('        <param name="pos" value="{0:.1f}"/>\n'.format(hgap))
    file.write('      </repeat>\n')
    file.write('    </paramrepeat>\n')
    file.write('  </macro>\n')

def setEnergy(file,energy):
    file.write('  <macro name="mv">\n')   
    file.write('    <paramrepeat name="motor_pos_list">\n') 
    file.write('      <repeat nr="1">')  
    file.write('        <param name="motor" value="mono_energy"/>\n')
    file.write('        <param name="pos" value="{0:.2f}"/>\n'.format(energy))
    file.write('      </repeat>\n')
    file.write('    </paramrepeat>\n')
    file.write('  </macro>\n')

def setGratingAngle(file,angle):
    file.write('  <macro name="mv">\n')   
    file.write('    <paramrepeat name="motor_pos_list">\n') 
    file.write('      <repeat nr="1">')  
    file.write('        <param name="motor" value="mono_gpit"/>\n')
    file.write('        <param name="pos" value="{0:.2f}"/>\n'.format(angle))
    file.write('      </repeat>\n')
    file.write('    </paramrepeat>\n')
    file.write('  </macro>\n')   

def setM2Angle(file,angle):
    file.write('  <macro name="mv">\n')   
    file.write('    <paramrepeat name="motor_pos_list">\n') 
    file.write('      <repeat nr="1">')  
    file.write('        <param name="motor" value="mono_m2pit"/>\n')
    file.write('        <param name="pos" value="{0:.2f}"/>\n'.format(angle))
    file.write('      </repeat>\n')
    file.write('    </paramrepeat>\n')
    file.write('  </macro>\n')   
    
def scanEnergy(file,start,end,numSteps):
    file.write('  <macro name="ascan">\n')   
    file.write('    <param name="motor" value="mono_energy"/>\n')
    file.write('    <param name="start_pos" value="{0:.1f}"/>\n'.format(start))
    file.write('    <param name="final_pos" value="{0:.1f}"/>\n'.format(end))
    file.write('    <param name="nr_interv" value="{0}"/>\n'.format(numSteps))
    file.write('    <param name="integ_time" value="0.5"/>\n')
    file.write('  </macro>\n')    

def scanEnergyPitchCorrect(file,start,end,numSteps):
    pitchStart=lookupM1Pitch('LookupTables/M1_pitch.txt',start)
    pitchEnd=lookupM1Pitch('LookupTables/M1_pitch.txt',end)
    file.write('  <macro name="a2scan">\n')   
    file.write('    <param name="motor1" value="mono_energy"/>\n')
    file.write('    <param name="start_pos1" value="{0:.1f}"/>\n'.format(start))
    file.write('    <param name="final_pos1" value="{0:.1f}"/>\n'.format(end))
    
    file.write('    <param name="motor2" value="m1_pitch"/>\n')
    file.write('    <param name="start_pos2" value="{0:.1f}"/>\n'.format(pitchStart))
    file.write('    <param name="final_pos2" value="{0:.1f}"/>\n'.format(pitchEnd))
    
    file.write('    <param name="nr_interv" value="{0}"/>\n'.format(numSteps))
    file.write('    <param name="integ_time" value="0.5"/>\n')
    file.write('  </macro>\n')    
        
def scanM1Pitch(file,start,end,numSteps):
    file.write('  <macro name="ascan">\n')   
    file.write('    <param name="motor" value="m1_pitch"/>\n')
    file.write('    <param name="start_pos" value="{0:.1f}"/>\n'.format(start))
    file.write('    <param name="final_pos" value="{0:.1f}"/>\n'.format(end))
    file.write('    <param name="nr_interv" value="{0}"/>\n'.format(numSteps))
    file.write('    <param name="integ_time" value="0.5"/>\n')
    file.write('  </macro>\n')   


def scanMonoBaffleVOffset(file,start,end,stepsize):
    file.write('  <macro name="ascan">\n')   
    file.write('    <param name="motor" value="mono_baff_v_offset"/>\n')
    file.write('    <param name="start_pos" value="{0:.1f}"/>\n'.format(start))
    file.write('    <param name="final_pos" value="{0:.1f}"/>\n'.format(end))
    file.write('    <param name="nr_interv" value="{0}"/>\n'.format(int(abs(end-start)/stepsize)+1))
    file.write('    <param name="integ_time" value="0.5"/>\n')
    file.write('  </macro>\n')    
   
def scanMonoBaffleHOffset(file,start,end,stepsize):
    file.write('  <macro name="ascan">\n')   
    file.write('    <param name="motor" value="mono_baff_h_offset"/>\n')
    file.write('    <param name="start_pos" value="{0:.1f}"/>\n'.format(start))
    file.write('    <param name="final_pos" value="{0:.1f}"/>\n'.format(end))
    file.write('    <param name="nr_interv" value="{0}"/>\n'.format(int(abs(end-start)/stepsize)+1))
    file.write('    <param name="integ_time" value="0.5"/>\n')
    file.write('  </macro>\n')    

-1835.5

***
## Generate a zero order M1 pitch measurement sequence


In [9]:
outputFileName = "zero_order_M1_pitch.xml"


gratingAngleStart = 6
gratingAngleStop = 18
gratingAngleStep = 1

numberOfGratingAngles = int(((gratingAngleStop-gratingAngleStart)/gratingAngleStep)+1)
print("This sequence will contain {0} grating angles".format(numberOfGratingAngles))

M1_pitch_start = -1790
M1_pitch_end = -1830
M1_pitch_stepsize = 0.5
M1_pitch_numScanPoints = abs(int(((M1_pitch_end-M1_pitch_start)/M1_pitch_stepsize)+1))

# By alternating the M1 scans between low-->high and high-->low we save some time
UP = 1
DOWN = 0
scanDirection = UP

f = open(outputFileName,'w')
f.write('<sequence>\n')

# Initialize the beamline
setEPUphase(f,0)
setEPUgap(f,65)
setFEMask_to_3pt5_mm(f)
centerMonoBaffles(f)
setMonoBaffleAperture(f,2)
setExitSlit(f,100,500)
setM4Diagnostick(f,79)

for angle in np.linspace(gratingAngleStart,gratingAngleStop,numberOfGratingAngles):
 
    setGratingAngle(f,angle)
    setM2Angle(f,angle)
    
    if scanDirection == UP:
        scanM1Pitch(f,M1_pitch_start,M1_pitch_end,M1_pitch_numScanPoints)
        scanDirection = DOWN
    else:
        scanM1Pitch(f,M1_pitch_end,M1_pitch_start,M1_pitch_numScanPoints)
        scanDirection = UP  
    
   
f.write('</sequence>')
f.close()

This sequence will contain 13 grating angles


***
## Generate a first order M1 pitch measurement sequence


In [10]:
from BLOCH_beamline import *

outputFileName = "first_order_M1_pitch.xml"

M1_pitch_start = -1750 #in units of fundamental peak fwhm, not eV!
M1_pitch_end = -1850
M1_pitch_stepsize = 0.5
M1_pitch_numScanPoints = abs(int(((M1_pitch_end-M1_pitch_start)/M1_pitch_stepsize)+1))

# By alternating the M1 scans between low-->high and high-->low we save some time
UP = 1
DOWN = 0
scanDirection = UP

f = open(outputFileName,'w')
f.write('<sequence>\n')

# Initialize the beamline
setEPUphase(f,0)
setFEMask_to_3pt5_mm(f)
centerMonoBaffles(f)
setMonoBaffleAperture(f,2)
setExitSlit(f,100,500)
setM4Diagnostick(f,79)

for hv in [16.8,18.5,20.3,22.5,25,28,31.8,36,41.5,48,56.6,67,80,100,127,166,225]:
 
    setEnergy(f,hv)
    setEPUgap(f,lookupGap(Path('LookupTables/HPOLfundamental.txt'),hv))
    
    if scanDirection == UP:
        scanM1Pitch(f,M1_pitch_start,M1_pitch_end,M1_pitch_numScanPoints)
        scanDirection = DOWN
    else:
        scanM1Pitch(f,M1_pitch_end,M1_pitch_start,M1_pitch_numScanPoints)
        scanDirection = UP  
    
   
f.write('</sequence>')
f.close()

In [9]:
outputFileName = "(2019.02.11)HPOL_fundamentals.xml"

# For this to work, we need to know where to look for the fundamental peaks. Here's an 
# approximately correct lookup table:
lookupTable = 'HPOLfundamental.txt'

gapStart = 18.0
gapStop = 90.0
gapStep = 1

numberOfGaps = int(((gapStop-gapStart)/gapStep)+1)
print("This sequence will contain {0} gaps".format(numberOfGaps))
# The fundamental peak widths are approximately hv/29
# That means that as we go up in EPU gap, the peak width will increase
# We should therefore make our energy scan range in units of peak FWHM instead
# of doing a fixed eV range.

scanWidthEitherSide = 3.1 #in units of fundamental peak fwhm, not eV!
scanNumPoints = 100

# By alternating the energy scans between low-->high and high-->low we save some 
# time waiting for the slow monochromator to move.
UP = 1
DOWN = 0
scanDirection = UP

f = open(outputFileName,'w')
f.write('<sequence>\n')

# Initialize the beamline
#setFEMask_to_3pt5mm(f)
#centerMonoBaffles(f)
#setM4Diagnostick(f,137)
setExitSlit(f,500,1200)
setMonoBaffleAperture(f,1)
#setEPUphase(f,0)

for EPU_gap in np.linspace(gapStart,gapStop,numberOfGaps):
    
    hv=lookupHarmonicEnergy(lookupTable='LookupTables/HPOLfundamental.txt',gap=EPU_gap,orderNumber=1)
    setM1Pitch(f,round(lookupM1Pitch(hv),1))
    hv_start = hv - (scanWidthEitherSide*hv/29)
    hv_stop = hv + (scanWidthEitherSide*hv/29)  
    
     # The monochromator energy is limited to 10eV at the moment. Don't try to scan below 10
    if hv_start<10:
        hv_start=10   
    
    setEPUgap(f,EPU_gap)  
    
    if scanDirection == UP:
        scanEnergy(f,hv_start,hv_stop,scanNumPoints)
        scanDirection = DOWN
    else:
        scanEnergy(f,hv_stop,hv_start,scanNumPoints)
        scanDirection = UP  
    
   
f.write('</sequence>')
f.close()

This sequence will contain 73 gaps


In [11]:
outputFileName = "(2019.02.11)HPOL_fundamentals_and_M1Pitch.xml"

#------ PART 1 (M1 pitch)----------
M1_pitch_start = -1780 #in units of fundamental peak fwhm, not eV!
M1_pitch_end = -1880
M1_pitch_stepsize = 2
M1_pitch_numScanPoints = abs(int(((M1_pitch_end-M1_pitch_start)/M1_pitch_stepsize)+1))

# By alternating the M1 scans between low-->high and high-->low we save some time
UP = 1
DOWN = 0
scanDirection = UP

f = open(outputFileName,'w')

f.write('<sequence>\n')

# Initialize the beamline
setEPUphase(f,0)
setFEMask_to_3pt5_mm(f)
centerMonoBaffles(f)
setMonoBaffleAperture(f,2)
setExitSlit(f,100,1200)
setM4Diagnostick(f,137)

for hv in [10.5,11,11.5,12,12.5,13,14,15,16,17,18,19,20,25,30,35,40,45,50,60,70,80,90,100,120,140,160,180]:
 
    setEnergy(f,hv)
    setEPUgap(f,lookupGap(Path('LookupTables/HPOLfundamental.txt'),hv))
    
    if scanDirection == UP:
        scanM1Pitch(f,M1_pitch_start,M1_pitch_end,M1_pitch_numScanPoints)
        scanDirection = DOWN
    else:
        scanM1Pitch(f,M1_pitch_end,M1_pitch_start,M1_pitch_numScanPoints)
        scanDirection = UP  


# -------- PART 2 ----------

# For this to work, we need to know where to look for the fundamental peaks. Here's an 
# approximately correct lookup table:
lookupTable = 'HPOLfundamental.txt'

gapStart = 18.0
gapStop = 90.0
gapStep = 1

numberOfGaps = int(((gapStop-gapStart)/gapStep)+1)
print("This sequence will contain {0} gaps".format(numberOfGaps))
# The fundamental peak widths are approximately hv/29
# That means that as we go up in EPU gap, the peak width will increase
# We should therefore make our energy scan range in units of peak FWHM instead
# of doing a fixed eV range.

scanWidthEitherSide = 3.1 #in units of fundamental peak fwhm, not eV!
scanNumPoints = 100

# By alternating the energy scans between low-->high and high-->low we save some 
# time waiting for the slow monochromator to move.
UP = 1
DOWN = 0
scanDirection = UP

# Initialize the beamline
#setFEMask_to_3pt5mm(f)
#centerMonoBaffles(f)
#setM4Diagnostick(f,137)
setExitSlit(f,700,1200)
setMonoBaffleAperture(f,1.7)
#setEPUphase(f,0)

for EPU_gap in np.linspace(gapStart,gapStop,numberOfGaps):
    
    hv=lookupHarmonicEnergy(lookupTable='LookupTables/HPOLfundamental.txt',gap=EPU_gap,orderNumber=1)
    setM1Pitch(f,round(lookupM1Pitch(hv),1))
    hv_start = hv - (scanWidthEitherSide*hv/29)
    hv_stop = hv + (scanWidthEitherSide*hv/29)  
    
     # The monochromator energy is limited to 10eV at the moment. Don't try to scan below 10
    if hv_start<10:
        hv_start=10   
    
    setEPUgap(f,EPU_gap)  
    
    if scanDirection == UP:
        scanEnergy(f,hv_start,hv_stop,scanNumPoints)
        scanDirection = DOWN
    else:
        scanEnergy(f,hv_stop,hv_start,scanNumPoints)
        scanDirection = UP  
    
   
f.write('</sequence>')
f.close()

This sequence will contain 73 gaps


***
## Measure the HPOL beam FWHM in realspace at the mono baffle position


In [11]:
outputFileName = "HPOL_fundamental_FWHM.xml"

# For this to work, we need to know where to look for the fundamental peaks. Here's an 
# approximately correct lookup table:
lookupTable = 'HPOLfundamental.txt'

gapStart = 19.0
gapStop = 87.0
gapStep = 1.0

numberOfGaps = int(((gapStop-gapStart)/gapStep)+1)

# All of these are in mm
scanRange = 5 #Total scan range is 2x this
scanCenter_horizOffset = 0.65 
scanCenter_vertOffset = -0.15 
baffleAperture = 0.25
scanStepSize = 0.05


# By alternating the energy scans between low-->high and high-->low we save some 
# time waiting for the slow monochromator to move.
UP = 1
DOWN = 0
scanDirection = UP

f = open(outputFileName,'w')
f.write('<sequence>\n')

# Initialize the beamline
centerMonoBaffles(f)
setM4Diagnostick(f,80)
setExitSlit(f,1200,530)
setMonoBaffleAperture(f,baffleAperture)

for EPU_gap in np.linspace(gapStart,gapStop,numberOfGaps):

    hv=lookupHarmonicEnergy(lookupTable=lookupTable,gap=EPU_gap,orderNumber=1)

    setGap(f,EPU_gap)
    setEnergy(f,hv)
    
    centerMonoBaffles(f)
    scanMonoBaffleVOffset(f,scanCenter_vertOffset-scanRange,scanCenter_vertOffset+scanRange,scanStepSize)
    centerMonoBaffles(f)
    scanMonoBaffleHOffset(f,scanCenter_horizOffset-scanRange,scanCenter_horizOffset+scanRange,scanStepSize)
    
f.write('</sequence>')
f.close()

***
## Generate a vertical polarization fundamental peak scan sequence

For each gap, measure the fundamental peak for 4 different mono baffle openings

In [8]:
outputFileName = "VPOL_fundamentals.xml"

# For this to work, we need to know where to look for the fundamental peaks. Here's an 
# approximately correct lookup table:
lookupTable = 'vpol_gaps.txt'

gapStart = 22.0
gapStop = 66.0
gapStep = 1.0

numberOfGaps = int(((gapStop-gapStart)/gapStep)+1)

# The fundamental peak widths are approximately hv/29
# That means that as we go up in EPU gap, the peak width will increase
# We should therefore make our energy scan range in units of peak FWHM instead
# of doing a fixed eV range.

scanWidthBelow = 3 #in units of fundamental peak fwhm, not eV!
scanWidthAbove = 3
scanNumPoints = 100

# By alternating the energy scans between low-->high and high-->low we save some 
# time waiting for the slow monochromator to move.
UP = 1
DOWN = 0
scanDirection = UP



f = open(outputFileName,'w')
f.write('<sequence>\n')
centerMonoBaffles(f)

for EPU_gap in np.linspace(gapStart,gapStop,numberOfGaps):
    
    hv=lookupHarmonicEnergy(lookupTable=lookupTable,gap=EPU_gap,orderNumber=1)
    hv_start = hv - (scanWidthBelow*hv/29)
    hv_stop = hv + (scanWidthBelow*hv/29)  
    
     # The monochromator energy is limited to 10eV at the moment. Don't try to scan below 10
    if hv_start<10:
        hv_start=10   
        
    setGap(f,EPU_gap)     
    
    for apertureSize in [1,2,4,6]:
        setMonoBaffleAperture(f,apertureSize)
        if scanDirection == UP:
            scanEnergy(f,hv-scanWidthBelow,hv+scanWidthAbove,scanNumPoints)
            scanDirection = DOWN
        else:
            scanEnergy(f,hv+scanWidthAbove,hv-scanWidthBelow,scanNumPoints)
            scanDirection = UP                  

f.write('</sequence>')
f.close()

## M1 pitch check

In [8]:
outputFileName = "HPOL_fundamental_M1Pitch.xml"

# For this to work, we need to know where to look for the fundamental peaks. Here's an 
# approximately correct lookup table:
lookupTable = 'HPOLfundamental.txt'

gapStart = 19.0
gapStop = 87.0
gapStep = 1.0

numberOfGaps = int(((gapStop-gapStart)/gapStep)+1)

# The fundamental peak widths are approximately hv/29
# That means that as we go up in EPU gap, the peak width will increase
# We should therefore make our energy scan range in units of peak FWHM instead
# of doing a fixed eV range.



# By alternating the energy scans between low-->high and high-->low we save some 
# time waiting for the slow monochromator to move.
UP = 1
DOWN = 0
scanDirection = UP

f = open(outputFileName,'w')
f.write('<sequence>\n')

for EPU_gap in np.linspace(gapStart,gapStop,numberOfGaps):
    
    hv=lookupHarmonicEnergy(lookupTable=lookupTable,gap=EPU_gap,orderNumber=1) 
    
     # The monochromator energy is limited to 10eV at the moment. Don't try to scan below 10
    if hv<10:
        hv=10   
    
    setEnergy(f,hv)
    setGap(f,EPU_gap)   
    scanM1Pitch(f,-1600,-1700,150)
    scanM1Pitch(f,-1600,-1700,150)     
    
f.write('</sequence>')
f.close()

***
## Generate a horizontal polarization third-harmonic peak scan sequence

For each gap, measure the third harmonic peak for 4 different mono baffle openings

In [10]:
outputFileName = "HPOL_thirdHarmonics.xml"

# For this to work, we need to know where to look for the fundamental peaks. Here's an 
# approximately correct lookup table:
lookupTable = 'HPOLfundamental.txt'

gapStart = 47.0
gapStop = 78
gapStep = 1.0

numberOfGaps = int(((gapStop-gapStart)/gapStep)+1)

# The fundamental peak widths are approximately hv/29

# ** I don't know if it's the same width for the thirds! Find out and correct this!***

# That means that as we go up in EPU gap, the peak width will increase
# We should therefore make our energy scan range in units of peak FWHM instead
# of doing a fixed eV range.

scanWidthBelow = 3 #in units of fundamental peak fwhm, not eV!
scanWidthAbove = 3
scanNumPoints = 100

# By alternating the energy scans between low-->high and high-->low we save some 
# time waiting for the slow monochromator to move.
UP = 1
DOWN = 0
scanDirection = UP

f = open(outputFileName,'w')
f.write('<sequence>\n')
centerMonoBaffles(f)

for EPU_gap in np.linspace(gapStart,gapStop,numberOfGaps):
    
    hv=lookupHarmonicEnergy(lookupTable=lookupTable,gap=EPU_gap,orderNumber=3)
    hv_start = hv - (scanWidthBelow*hv/29)
    hv_stop = hv + (scanWidthBelow*hv/29)   
        
    setGap(f,EPU_gap)     
    
    for apertureSize in [1,2,4,6]:
        setMonoBaffleAperture(f,apertureSize)
        if scanDirection == UP:
            scanEnergy(f,hv-scanWidthBelow,hv+scanWidthAbove,scanNumPoints)
            scanDirection = DOWN
        else:
            scanEnergy(f,hv+scanWidthAbove,hv-scanWidthBelow,scanNumPoints)
            scanDirection = UP                  

f.write('</sequence>')
f.close()